<a href="https://colab.research.google.com/github/aproca/MNIST_To_CIFAR10AdvRep/blob/master/Encoded_MNIST_trained_on_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
src = list(files.upload().values())[0]

Saving mnist_language_test (2).csv to mnist_language_test (2).csv


In [0]:
from sklearn.model_selection import train_test_split
import torch
from torch.autograd import Variable
import numpy as np
import os
import io
import requests
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset, sampler
from torch import LongTensor, DoubleTensor, FloatTensor

class ChunkSampler(sampler.Sampler):
   """Samples elements sequentially from some offset.
   Arguments:
       num_samples: # of desired datapoints
       start: offset where we should start selecting from
   """
   def __init__(self, num_samples, start = 0):
       self.num_samples = num_samples
       self.start = start
   def __iter__(self):
       return iter(range(self.start, self.start + self.num_samples))
   def __len__(self):
       return self.num_samples
NUM_TRAIN = 100
NUM_TEST = 100

#datatrain = pd.read_csv("mnist_ascii_70000_2 (2).csv")
datatrain = pd.read_csv("mnist_language_test (2).csv")

#datatrain = datatrain[0:10]

y = datatrain['784'].values              # number labels
                               # removes number labels and image number label
X = datatrain.drop(columns=['784']).values

BATCH_SIZE = 16

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

print(len(X_train))
X_train = X_train.reshape(8499,1,28,28)
X_test = X_test.reshape(1500,1,28,28)


# data type is long
torch_X_train = torch.from_numpy(X_train).type(torch.FloatTensor) # image arrays
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # label array

# # create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

dataset = TensorDataset( FloatTensor(torch_X_train), LongTensor(torch_y_train) )

loader = DataLoader(dataset, batch_size= BATCH_SIZE ,sampler=ChunkSampler(NUM_TRAIN,0), shuffle = False)

dataset2 = TensorDataset(FloatTensor(torch_X_test),LongTensor(torch_y_test))
loader2 = DataLoader(dataset2, batch_size = BATCH_SIZE, shuffle = False)

# Pytorch train and test sets
#train = torch.utils.data.TensorDataset(torch_X_train, torch_y_train)
#test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,
                                          sampler=ChunkSampler(NUM_TRAIN,0), num_workers=8)
#train_loader = torch.utils.data.DataLoader(dataset, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size = BATCH_SIZE, sampler=ChunkSampler(NUM_TEST,0), shuffle = False)


8499


In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

# Hyper parameters
num_epochs = 50
num_classes = 10
batch_size = 16
learning_rate = 0.001

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
     
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


cuda
Epoch [1/50], Step [2/7], Loss: 2.2804
Epoch [1/50], Step [4/7], Loss: 2.6117
Epoch [1/50], Step [6/7], Loss: 2.6744
Epoch [2/50], Step [2/7], Loss: 2.3711
Epoch [2/50], Step [4/7], Loss: 2.4919
Epoch [2/50], Step [6/7], Loss: 2.2542
Epoch [3/50], Step [2/7], Loss: 2.3184
Epoch [3/50], Step [4/7], Loss: 2.3734
Epoch [3/50], Step [6/7], Loss: 2.2282
Epoch [4/50], Step [2/7], Loss: 2.0648
Epoch [4/50], Step [4/7], Loss: 2.2788
Epoch [4/50], Step [6/7], Loss: 2.2380
Epoch [5/50], Step [2/7], Loss: 2.0276
Epoch [5/50], Step [4/7], Loss: 2.2147
Epoch [5/50], Step [6/7], Loss: 2.0743
Epoch [6/50], Step [2/7], Loss: 1.9542
Epoch [6/50], Step [4/7], Loss: 2.0667
Epoch [6/50], Step [6/7], Loss: 1.7984
Epoch [7/50], Step [2/7], Loss: 1.6382
Epoch [7/50], Step [4/7], Loss: 1.7815
Epoch [7/50], Step [6/7], Loss: 1.4353
Epoch [8/50], Step [2/7], Loss: 1.3124
Epoch [8/50], Step [4/7], Loss: 1.4032
Epoch [8/50], Step [6/7], Loss: 1.0581
Epoch [9/50], Step [2/7], Loss: 1.0077
Epoch [9/50], Step [

KeyboardInterrupt: ignored

In [0]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        print(images.size())
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

torch.Size([16, 1, 28, 28])
torch.Size([16, 1, 28, 28])
torch.Size([16, 1, 28, 28])
torch.Size([16, 1, 28, 28])
torch.Size([16, 1, 28, 28])
torch.Size([16, 1, 28, 28])
torch.Size([4, 1, 28, 28])
Test Accuracy of the model on the 10000 test images: 68.0 %
